### Importing

In [105]:
import squigglepy as sq
import numpy as np
from squigglepy.numbers import K, M, B
import pickle
import os
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

params = pickle.load(open(os.path.join('input_data', "Sentience Parameters.p"), 'rb'))
NUM_SCENARIOS = params['N_SCENARIOS']

### Get species per category and species proxies

In [106]:
## Get proxies for moral weights

SPECIES = ['pigs', 'chickens', 'carp', 'octopuses', 'bees', 'salmon', 'crayfish', 'shrimp',  'crabs', 'bsf', 'silkworms']

species_in_category_dict = {'wild vertebrates': ['all vertebrates'], 
                            'farmed fishes': ['fish'], 
                            'farmed aquatic invertebrates': ['shrimp', 'crayfish, crabs, lobsters'],
                            'farmed land invertebrates': ['silkworms', 'crickets', 'cochineals', 'bsf', 'mealworms'],
                            'farmed chickens': ['chickens'],
                            'farmed animals': ['fish', 'shrimp', 'crayfish, crabs, lobsters', 
                                               'silkworms', 'crickets', 'cochineals', 'bsf', 'mealworms', 
                                               'chickens', 'cows', 'pigs', 'amphibians and reptiles', 'rabbits', 'mink, foxes, racoon dogs, rabbits, chinchillas',
                                               'ducks, geese'],
                            'farmed cows': ['cows'],
                            'farmed pigs': ['pigs'],
                            'farmed rabbits': ['rabbits'],
                            'farmed animals for fur': ['mink, foxes, racoon dogs, rabbits, chinchillas'],
                            'farmed animals for foie gras': ['ducks, geese'],
                            'animals in labs': ['rabbits, mice, dogs, monkeys'], 
                            'companion animals': ['dogs, cats'],
                            'working animals': ['horses, donkeys, mules, dogs'],
                            'animals in entertainment': ['8600 species in zoos/aquariums']}

proxy_species_and_weights = {'all vertebrates': {'carp': 0.5, 'salmon': 0.5},
                             'fish': {'carp': 0.5, 'salmon': 0.5},
                             'shrimp': {'shrimp': 1},
                            'crayfish, crabs, lobsters': {'crayfish': 0.5, 'crabs': 0.5},
                            'silkworms': {'silkworms': 1},
                            'crickets': {'bsf': 1},
                            'cochineals': {'silkworms': 1},
                            'bsf': {'bsf': 1},
                            'mealworms': {'silkworms': 1},
                            'chickens': {'chickens': 1},
                            'pigs': {'pigs': 1},
                            'amphibians and reptiles': {'chickens': 0.5, 'carp': 0.25, 'salmon':0.25},
                            'ducks, geese': {'chickens': 1},
                            'rabbits': {'pigs': 1},
                            'cows': {'pigs': 1},
                            'mink, foxes, racoon dogs, rabbits, chinchillas': {'pigs': 1},
                            'rabbits, mice, dogs, monkeys': {'pigs': 1},
                            'ducks, geese': {'chickens': 1},
                            'dogs, cats': {'pigs': 1},
                            'horses, donkeys, mules, dogs': {'pigs': 1},
                            '8600 species in zoos/aquariums': {'pigs': 1/3, 'chickens': 1/3, 'carp': 1/6, 'salmon': 1/6}}


### Set population counts

In [107]:
population_models = {'all vertebrates': sq.norm(10**13, 10**15, lclip=10**13, rclip=10**15),
               'fish': sq.lognorm(6*10**10, 2*10**11, lclip=4*10**10, rclip=3*10**11), 
               'shrimp': sq.norm(1.8*10**11, 3.20*10**11, lclip=1.2*10**11, rclip=4*10**11),
               'crayfish, crabs, lobsters': sq.norm(2.7*10**10, 5.1*10**10, lclip=2*10**10, rclip=7.5*10**10),
               'silkworms': sq.norm(4*10**10, 1*10**11, lclip=2*10**10, rclip=1.2*10**11),
               'crickets': sq.lognorm(1*10**9, 2*10**10, lclip=1*10**9, rclip=2*10**10),
               'cochineals': sq.norm(1*10**11, 3*10**11, lclip= 4*10**10, rclip=4*10**11),
               'bsf': sq.lognorm(9*10*6, 2.6*10**7, lclip=5*10**6, rclip=5*10**7),
               'mealworms': sq.lognorm(1*10**10, 1*10**11, lclip=1.5*10**9, rclip=1.4*10**11),
               'chickens': sq.norm(1.8*10**10, 2.5*10**10, lclip=1.5*10**10, rclip=2.8*10**10),
               'amphibians and reptiles': sq.lognorm(5.8*10**8, 3.5*10**9, lclip=3.3*10**8, rclip=5.3*10**9),
               'cows': sq.norm(1.2*10**9, 1.8*10**9, lclip=1*10**9, rclip=2*10**9),
               'pigs': sq.norm(9*10**8, 1.06*10**9, lclip= 8.5*10**8, rclip=2*10**9),
               'rabbits': sq.norm(2.5*10**8, 3.5*10**8, lclip=2.2*10**8, rclip=3.8*10**8),
               'mink, foxes, racoon dogs, rabbits, chinchillas': sq.norm(2.5*10**7, 6.5*10**7, lclip=1.5*10**7, rclip=8*10**7),
               'ducks, geese': sq.norm(1.1*10**7, 1.9*10**7, lclip=5*10**6, rclip=2.5*10**7),
               'rabbits, mice, dogs, monkeys': sq.norm(1.1*10**8, 1.8*10**8, lclip=8.2*10**7, rclip=2.1*10**8),
               'dogs, cats': sq.norm(1.2*10**9, 1.8*10**9, lclip=9*10**8, rclip=2.1*10**9),
               'horses, donkeys, mules, dogs': sq.norm(2*10**8, 3*10**8, lclip=1*10**8, rclip=4*10**8),
               '8600 species in zoos/aquariums': sq.norm(1*10**6, 1*10**7, lclip=5*10**5, rclip=1.2*10**7)

}

def generate_populations():
    populations = {}
    for animal, model in population_models.items():
        pop_array = sq.sample(model, NUM_SCENARIOS)
        populations[animal] = pop_array
    return populations

populations = generate_populations()

### Moral weight weighted averages by category

In [108]:
def load_welfare_ranges(species):
    with open(os.path.join('sent_adj_welfare_ranges', '{}_wr_{}.p'.format(species, 'Mixture Neuron Count')), 'rb') as f_wr:
        species_wr = list(pickle.load(f_wr)) 
    return species_wr

def one_animal_weighted_wr(animal):
    animal_proxies = proxy_species_and_weights[animal]

    animal_wr_arr = np.zeros(NUM_SCENARIOS)
    for proxy_species, proxy_species_weight in animal_proxies.items():
        #print('proxy species: {}'.format(proxy_species))
        #print('weight: {}'.format(proxy_species_weight))
        species_wr_arr = np.array(load_welfare_ranges(proxy_species))
        #print('Before: {}'.format(animal_wr_arr[:10]))
        animal_wr_arr += species_wr_arr*proxy_species_weight
        #print('After: {}'.format(species_wr_arr[:10]))

    #print(sq.get_percentiles(animal_wr_arr))
    return animal_wr_arr

def get_relative_population(category):
    category_species_lst = species_in_category_dict[category]
    category_population = np.zeros(NUM_SCENARIOS)

    relative_populations = {}

    for species in category_species_lst:
        species_pop = populations[species]
        category_population += species_pop
    
    for species in category_species_lst:
        rel_species_pop = populations[species]/category_population
        relative_populations[species] = rel_species_pop
    
    return relative_populations

def one_category_weighted_welfare_range(category):
    category_species_lst = species_in_category_dict[category]
    category_relative_populations = get_relative_population(category)

    moral_weights = np.zeros(NUM_SCENARIOS)

    for animal in category_species_lst:
        animal_wr_weighted = one_animal_weighted_wr(animal)
        #print("Animal: {}".format(animal))
        #print("Moral weight: {}".format(animal_wr_weighted[:10]))
        animal_rel_pop = category_relative_populations[animal]
        #print("Rel pop: {}".format(animal_rel_pop[:10]))
        pop_weighted_moral_weight = animal_wr_weighted*animal_rel_pop
        #print("MW weighted: {}".format(pop_weighted_moral_weight[:10]))
        moral_weights += pop_weighted_moral_weight

    #print(sq.get_percentiles(moral_weights))
    return moral_weights

## All Category Moral Weights

In [109]:
categories = list(species_in_category_dict.keys())
print(categories)

def all_category_moral_weights():
    all_moral_weights = {}
    for category in categories:
        cat_mw = one_category_weighted_welfare_range(category)
        all_moral_weights[category] = cat_mw
        #print(category)
        #print(cat_mw[:10])
    return all_moral_weights

print(all_category_moral_weights())

['wild vertebrates', 'farmed fishes', 'farmed aquatic invertebrates', 'farmed land invertebrates', 'farmed chickens', 'farmed animals', 'farmed cows', 'farmed pigs', 'farmed rabbits', 'farmed animals for fur', 'farmed animals for foie gras', 'animals in labs', 'companion animals', 'working animals', 'animals in entertainment']
{'wild vertebrates': array([0.12440373, 0.43693668, 0.43514365, ..., 0.29044986, 0.31258747,
       0.23863012]), 'farmed fishes': array([0.12440373, 0.43693668, 0.43514365, ..., 0.29044986, 0.31258747,
       0.23863012]), 'farmed aquatic invertebrates': array([0.0083051 , 0.04588571, 0.00910003, ..., 0.03708706, 0.0348666 ,
       0.01469343]), 'farmed land invertebrates': array([3.63362891e-04, 1.03324990e-02, 6.05321700e-04, ...,
       2.06479692e-02, 3.93318599e-01, 1.36919445e-03]), 'farmed chickens': array([1.66727211, 0.65195858, 0.37867387, ..., 1.20095927, 0.72160867,
       0.84479331]), 'farmed animals': array([0.08562998, 0.10875568, 0.10342266, ...

## Make Data Frame

In [110]:
def make_dataframe(moral_weights):
    means = []
    fifth_percentiles = []
    medians = []
    ninty_fifth_percentiles = []
    species_adj_wrs = []

    for category in categories: 
        cat_mw = moral_weights[category]
        means.append(round(np.mean(cat_mw),3))
        fifth_percentiles.append(round(np.percentile(cat_mw, 5),3))
        medians.append(round(np.percentile(cat_mw, 50),3))
        ninty_fifth_percentiles.append(round(np.percentile(cat_mw, 95),3))

    cols = ["Mean","5th-pct", "50th-pct", "95th-pct"]
    dataframe = pd.DataFrame(list(zip(means, fifth_percentiles, medians, ninty_fifth_percentiles)), \
        columns=cols, index=categories)
    dataframe = dataframe.sort_values("50th-pct", ascending=False)
    path = os.path.join("sent_adj_welfare_ranges", "By Category Welfare Ranges - Indexed to Pigs - Summary Statistics.csv")
    dataframe.to_csv(path, index_label="Category")

    return dataframe
mws = all_category_moral_weights()
print(make_dataframe(mws))


                               Mean  5th-pct  50th-pct  95th-pct
farmed cows                   1.000    1.000     1.000     1.000
farmed pigs                   1.000    1.000     1.000     1.000
farmed rabbits                1.000    1.000     1.000     1.000
farmed animals for fur        1.000    1.000     1.000     1.000
animals in labs               1.000    1.000     1.000     1.000
companion animals             1.000    1.000     1.000     1.000
working animals               1.000    1.000     1.000     1.000
farmed chickens               0.645    0.317     0.636     1.031
farmed animals for foie gras  0.645    0.317     0.636     1.031
animals in entertainment      0.629    0.486     0.617     0.814
wild vertebrates              0.241    0.031     0.188     0.634
farmed fishes                 0.241    0.031     0.188     0.634
farmed animals                0.155    0.042     0.101     0.453
farmed aquatic invertebrates  0.215    0.007     0.075     0.944
farmed land invertebrates